# Hailee Kiesecker
# PS2
# Due: 3/13/2021

### Colab : None

Question 1: 

Implement the fit and predict procedures for the logistic regression (scikit is not allowed) with norm 2
regularization function (and Lambda parameter). Use as the input parameters of the gradient ascent the maximum
number of iterations (just a constant e.g. 100) and the learning factor (e.g. 0.01).



In [ ]:
import numpy as np 
import pandas as pd
import statsmodels.regression.linear_model as stats_model 
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,precision_recall_fscore_support,accuracy_score, precision_score, recall_score, f1_score 
from sklearn.model_selection import StratifiedKFold,KFold
# import data
from sklearn import datasets
# iris = datasets.load_iris()
# iris.data[:,:]

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm



# Sigmoid function (S(x)), e = Euler's number
$ S(x) = \frac {1} {1+e^{-x}}$





In [ ]:
class logistic_regression(object):

  def __init__(__self__, C, learning_factor=0.001, max_iter=100, tolerance = 0.1):
        __self__.learning_factor = learning_factor
        __self__.max_iter = max_iter
        __self__.C = C
        __self__.tolerance = tolerance

  def sigmoid(__self__, z):
    return 1 / (1+ np.exp(-z))

  #first we fit the model
  # will take in the features and target
  def fit(__self__, X, y):
        __self__.weights = np.zeros(X.shape[1]+1)
        X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)

        for _ in range(__self__.max_iter):
            dotted_X = np.dot(X, __self__.weights)
            y_pred = __self__.sigmoid(dotted_X)
            error = y - y_pred
            Num_features = X.shape[1]

            #always will be L2

            ############## Gradient decent function ############
            # SUM of all the weights plus learning factor * (C * X TRANSPOSED) 
            ####################################################
            dotted_XT = np.dot(X.T, error)
            gradient = (np.sum(__self__.weights) + __self__.learning_factor * ((__self__.C * (dotted_XT))))

            if np.all(abs(gradient) >= __self__.tolerance):
                __self__.weights -= gradient / Num_features
            else:
                break
                
        return __self__
  
  # predict 
  def predict(__self__, X):
    dotted_X = np.dot(X, __self__.weights[1:])
    roundM = __self__.sigmoid((dotted_X) + __self__.weights[0])
    roundM = np.round(roundM)
    return roundM

  #if __name__ == "__init__":
    #init()

In [ ]:

#I think this also does it but does not take in a lambda value
x = np.arange(10).reshape(-1, 1)
y = np.array([0, 1, 0, 0, 1, 1, 1, 1, 1, 1])
x = sm.add_constant(x)

model = sm.Logit(y, x)
result = model.fit(method="newton")
result.predict(x)

#(result.predict(x) >= 0.5).astype(int)

#result.pred_table()

#result.summary()

Optimization terminated successfully.
         Current function value: 0.350471
         Iterations 7


array([0.12208792, 0.24041529, 0.41872657, 0.62114189, 0.78864861,
       0.89465521, 0.95080891, 0.97777369, 0.99011108, 0.99563083])

Question 2: 

Using the iris dataset (the binary class, Iris Setosa vs others), K-fold cross validation, metrics (accuracy,
precision, recall, F1-score) and logistic regression (Question1) with L2 regularization, estimate the best
parameter C (the inverse of lambda).

In [ ]:
#iris = datasets.load_iris()
X, y = datasets.load_iris(return_X_y=True)

In [ ]:
X = iris.data
# if setisa 1 else 0
y = np.where(y == 0, 1, 0)
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.45)
scaler = StandardScaler()

X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

C=[0.001,0.005,0.01,0.05,0.1,0.5,1, 1.62 ,5,10,50,100,500,1000,1000,10000000]
classifier = LogisticRegression(random_state=0, class_weight='balanced',penalty='l2')
bestC = 0
bestValue = -1
cv = 5
skf = KFold(n_splits=cv)
for c in C:
    avgF1 = 0
    for train_index, test_index in skf.split(X_train, y_train):
        X_train1=X_train[train_index]
        X_val=X_train[test_index]
        
        y_train1=y_train[train_index]
        y_val=y_train[test_index]

        clf=LogisticRegression(random_state=0, class_weight='balanced',C=c,penalty='l2') # mine has an overflow error
        clf.fit(X_train1,y_train1)
        y_pred_val=clf.predict(X_val)
        avgF1+=f1_score(y_val, y_pred_val, average='macro')


    avgF1=avgF1/cv

    if bestValue<avgF1:
        bestValue=avgF1
        bestC=c
        classifier=clf
print('best C: ',bestC) 


best C:  50


In [ ]:
model = logistic_regression(1.6)
model.fit(X,y)

In [ ]:
model.predict(X)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression


X, y = load_iris(return_X_y=True)

max = 1000
perm_c = 0
perm_z = 0
insert_c = 0.01

for i in range(max):
  clf = LogisticRegression(random_state=0, C = insert_c).fit(X, y)
  clf.predict(X[:2, :])
  clf.predict_proba(X[:2, :])
  
  z1 = clf.score(X, y)

  #if score is better than previous store perm z  
  if (perm_z < z1):
    perm_c = insert_c
    perm_z = z1 

  insert_c = insert_c + 0.01


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

In [ ]:
perm_c

1.6200000000000012

# Reflection: 
Total coding time around 5 hours research time an additional 2-3. While I believe I answered the questions asked of me in some way the produced code is incorrect in my opinion. It was on the last night that I realized my inital solution to question 2 was wrong. Initially I had coded just a regular k-split and logistic regression seperatly to find the "best" C value. Then I relized the question was asking for them together. Looking for motivation I found the ksplit-logistic regression notebook and just revised that. I find my original solution of just using logistic regression in a for loop iterating over C values increasing by 0.01 a better estimate than the logistic regression with k split. This is mainly due to the fact that when ran on seperate occasions my for loop iteration always produces 1.62 while the k-split with logistic regression always produces a different optimal C value.